In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

sns.set_theme()

### Functions

In [65]:
# split a univariate sequence into samples
def split_sequence(sequence, window_size, step):
    X = list()
    for i in range(0, len(sequence), step):
        # find the end of this pattern
        end_ix = i + window_size
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)

def generate_pairs(data, labels, window_size, n_features):
    ### GENERATE PAIR SAMPLES FOR SIAMESE NETWORK ###

    os.environ['PYTHONHASHSEED'] = str(33)
    np.random.seed(33)
    random.seed(33)

    left_input = []
    right_input = []
    targets = []
    pairs = 3

    for i in range(len(data)):
        for _ in range(pairs):
            compare_to = i
            while compare_to == i:
                compare_to = random.randint(0,len(data)-1)
            left_input.append(data[i])
            right_input.append(data[compare_to])
            # targets.append(euclidean(data[i], data[compare_to]))
            if 1 in labels[i] and 1 in labels[compare_to]:
                targets.append(0.)
            elif 1 in labels[i] or 1 in labels[compare_to]:
                targets.append(1.)
            else:
                targets.append(0.)

    left_input = np.asarray(left_input).reshape(-1, window_size, n_features)
    right_input = np.asarray(right_input).reshape(-1, window_size, n_features)
    targets = np.asarray(targets)
    return left_input, right_input, targets

def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def SiamesNet():
    
    set_seed(33)
    
    left_input = Input((window_size, n_features))
    right_input = Input((window_size, n_features))

    lstmnet = Sequential([
        BatchNormalization(),
        LSTM(64, activation='relu', return_sequences=True, 
             input_shape=(window_size, n_features)),
        LSTM(16, activation='relu')
    ])

    encoded_l = lstmnet(left_input)
    encoded_r = lstmnet(right_input)

    L1_layer = Lambda(lambda tensor: K.abs(tensor[0] - tensor[1]))

    L1_distance = L1_layer([encoded_l, encoded_r])
    drop = Dropout(0.2)(L1_distance)
    prediction = Dense(1,activation='sigmoid')(drop)
    model = Model(inputs=[left_input,right_input],outputs=prediction)
    
    return model

### Pre-processing

In [66]:
dfs = []
for i in range(1, 68):
    dfs.append(pd.read_csv('./data/A1Benchmark/real_{}.csv'.format(i), index_col='timestamp'))

In [67]:
window_size = 12
step = window_size
n_features = 1
windows = split_sequence(dfs[8].value, window_size, step)
labels = split_sequence(dfs[8].is_anomaly, window_size, step)

In [68]:
train_windows, test_windows, train_labels, test_labels = train_test_split(windows, labels)

In [70]:
train_left_input, train_right_input, train_targets = generate_pairs(train_windows, train_labels, window_size, n_features)
test_left_input, test_right_input, test_targets = generate_pairs(test_windows, test_labels, window_size, n_features)

### Modeling

In [84]:
siamese_net = SiamesNet()
siamese_net.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

siamese_net.fit([train_left_input, train_right_input], train_targets, validation_split=0.15, epochs=50)

Epoch 1/50
8/8 [==============================] - 4s 76ms/step - loss: 0.6782 - accuracy: 0.9214 - val_loss: 0.6880 - val_accuracy: 0.9512
Epoch 2/50
8/8 [==============================] - 0s 11ms/step - loss: 0.6457 - accuracy: 0.9476 - val_loss: 0.6826 - val_accuracy: 0.9512
Epoch 3/50
8/8 [==============================] - 0s 11ms/step - loss: 0.5750 - accuracy: 0.9607 - val_loss: 0.6740 - val_accuracy: 0.9512
Epoch 4/50
8/8 [==============================] - 0s 11ms/step - loss: 0.4081 - accuracy: 0.9563 - val_loss: 0.6523 - val_accuracy: 0.9512
Epoch 5/50
8/8 [==============================] - 0s 12ms/step - loss: 0.3734 - accuracy: 0.9520 - val_loss: 0.6570 - val_accuracy: 0.9512
Epoch 6/50
8/8 [==============================] - 0s 11ms/step - loss: 0.3346 - accuracy: 0.9563 - val_loss: 0.6561 - val_accuracy: 0.9512
Epoch 7/50
8/8 [==============================] - 0s 10ms/step - loss: 0.3030 - accuracy: 0.9738 - val_loss: 0.6467 - val_accuracy: 0.9512
Epoch 8/50
8/8 [===========

### Predictions

In [110]:
y_pred = siamese_net.predict([test_left_input, test_right_input])

3/3 [==============================] - 0s 4ms/step


In [123]:
print(classification_report(test_targets, y_pred))

ValueError: Classification metrics can't handle a mix of binary and continuous targets